In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras_tuner
from tensorflow.keras import layers
import keras.backend as K

In [3]:
data_in = np.load("Data/images.npy")
data_out = np.load("Data/labels.npy")

In [4]:
def convert_to_num(hour, minute):
    num = hour+minute/60
    return num

def group(num):
    array = np.zeros(48)
    category = num//0.25
    category = int(category)
    array[category]=1
    return array
    
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]



In [5]:
def clock_loss_class_mae(y_true, y_pred):
    absdiff = K.abs(y_pred - y_true)
    norm_c = absdiff <= 6
    norm_c = tf.cast(norm_c, tf.float32)
    round_c = absdiff > 6
    round_c = tf.cast(round_c, tf.float32)
    normdiff = K.abs(y_pred - y_true)
    rounddiff = 12 - K.abs((y_pred - y_true))
    
    diff = norm_c * normdiff + round_c * rounddiff
    mae = K.mean(K.abs(diff))
    return mae

In [6]:
def build_model_cnn(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_fixed_filter',default=96, min_value=32, max_value=128, step=10),
        # adding kernel size
        kernel_size=hp.Choice('conv_fixed_kernel', values = [3,5,7]),
        #activation function
        activation='relu',
        input_shape=[150,150,1],
        padding='same',)
    )
    model.add(
    layers.MaxPooling2D(pool_size = hp.Choice('pool_size', values = [2,3,4]),
    ),
    )
    # Tune the number of layers.
    for i in range(hp.Int("num_cnn_layers",default=4, min_value=1, max_value=4)):
        if i == 0:
            model.add(
                layers.Conv2D(
                    #adding filter 
                    filters=hp.Int(f'conv_{i+1}_filter',default=256, min_value=32, max_value=512, step=16),
                    # adding kernel size
                    kernel_size=hp.Choice(f'conv_{i+1}_kernel', values = [3,5]),
                    #activation function
                    activation='relu',
                    padding='same'),
            )

        elif i == 1:
            model.add(
                layers.Conv2D(
                    #adding filter 
                    filters=hp.Int(f'conv_{i+1}_filter',default=348, min_value=32, max_value=512, step=16),
                    # adding kernel size
                    kernel_size=hp.Choice(f'conv_{i+1}_kernel', values = [3,5]),
                    #activation function
                    activation='relu',
                    padding='same'),
            )

        elif i == 2:
            model.add(
                layers.Conv2D(
                    #adding filter 
                    filters=hp.Int(f'conv_{i+1}_filter',default=348, min_value=32, max_value=512, step=16),
                    # adding kernel size
                    kernel_size=hp.Choice(f'conv_{i+1}_kernel', values = [3,5]),
                    #activation function
                    activation='relu',
                    padding='same'),
            )

        else:
            model.add(
                layers.Conv2D(
                    #adding filter 
                    filters=hp.Int(f'conv_{i+1}_filter',default=256, min_value=32, max_value=512, step=16),
                    # adding kernel size
                    kernel_size=hp.Choice(f'conv_{i+1}_kernel', values = [3,5]),
                    #activation function
                    activation='relu',
                    padding='same'),
            )

        model.add(
            layers.MaxPooling2D(
                pool_size = 2
            ),
            )
                
    model.add(layers.Flatten())        

    for i in range(hp.Int("num_dense_layers", default=2, min_value=1, max_value=3)):
        if i == 0:
            model.add(
                layers.Dense(
                    units=hp.Int(f"units_{i+1}",default=128, min_value=60, max_value=515, step=20),
                    activation=hp.Choice("activation", ["relu", "tanh"]),),
            )
        
        elif i == 1:
            model.add(
                layers.Dense(
                    units=hp.Int(f"units_{i+1}",default=60, min_value=60, max_value=515, step=20),
                    activation=hp.Choice("activation", ["relu", "tanh"]),),
            )

        else:
            model.add(
                layers.Dense(
                    units=hp.Int(f"units_{i+1}",default=60, min_value=60, max_value=515, step=20),
                    activation=hp.Choice("activation", ["relu", "tanh"]),),
            )

        if hp.Boolean(f"dropout_{i+1}", default=True):
            model.add(layers.Dropout(rate=hp.Choice(f'rate_{i+1}', default=0.25, values = [0.25,0.5,0.75])))
    
    model.add(layers.Dense(48, activation="softmax"))

    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss="categorical_crossentropy", 
        metrics=["accuracy", clock_loss_class_mae],
    )
    return model

build_model_cnn(keras_tuner.HyperParameters())

In [7]:
data_in, data_out = unison_shuffled_copies(data_in, data_out)

In [8]:
class_out = []
for i in range(len(data_out)):
    hour = data_out[i][0]
    minute = data_out[i][1]
    hr_min_num = convert_to_num(hour, minute)
    hr_min_class = group(hr_min_num)
    class_out.append(hr_min_class)
class_out = np.array(class_out)

In [9]:
X_test, X_train, X_valid = data_in[12000:15000]/np.amax(data_in), data_in[:12000]/np.amax(data_in), data_in[15000:]/np.amax(data_in)
y_test, y_train, y_valid = class_out[12000:15000], class_out[:12000], class_out[15000:]

In [12]:
hp = keras_tuner.HyperParameters()

tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model_cnn,
    objective="val_accuracy",
    max_trials=30,
    num_initial_points=2,
    seed=90,
    directory="Task2",
    project_name="CNN3",
    #overwrite=True,
    #executions_per_trial,
)

INFO:tensorflow:Reloading Oracle from existing project Task2\CNN3\oracle.json
INFO:tensorflow:Reloading Tuner from Task2\CNN3\tuner0.json


In [166]:
tuner.search_space_summary()


Search space summary
Default search space size: 20
conv_fixed_filter (Int)
{'default': 96, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 10, 'sampling': None}
conv_fixed_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5, 7], 'ordered': True}
pool_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 4], 'ordered': True}
num_cnn_layers (Int)
{'default': 4, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': None}
conv_1_filter (Int)
{'default': 256, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
conv_2_filter (Int)
{'default': 256, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 16, 'sampling': None}
conv_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
conv_3_filter (Int)
{'default': 348, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 16,

In [167]:
tuner.search(X_train, y_train, epochs=400, validation_data=(X_valid, y_valid),
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=25)], batch_size=300,use_multiprocessing=True, workers=15)


Trial 1 Complete [00h 18m 02s]
val_accuracy: 0.7576666474342346

Best val_accuracy So Far: 0.7576666474342346
Total elapsed time: 00h 18m 02s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
122               |52                |conv_fixed_filter
3                 |3                 |conv_fixed_kernel
3                 |3                 |pool_size
1                 |4                 |num_cnn_layers
144               |288               |conv_1_filter
5                 |3                 |conv_1_kernel
432               |208               |conv_2_filter
3                 |3                 |conv_2_kernel
368               |208               |conv_3_filter
5                 |5                 |conv_3_kernel
272               |96                |conv_4_filter
3                 |3                 |conv_4_kernel
2                 |1                 |num_dense_layers
80                |500               |units_1
tanh              |relu              |activation

KeyboardInterrupt: 

In [13]:
# Get the top 3 models.
models_cnn_class = tuner.get_best_models(num_models=3)
best_model_cnn_class = models_cnn_class[0]
# Build model
best_model_cnn_class.build(input_shape=(None, 28, 28))
best_model_cnn_class.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 52)      520       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 52)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 288)       135072    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 288)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 208)       539344    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 208)      0

In [14]:
tuner.results_summary(num_trials = 1)

Results summary
Results in Task2\CNN3
Showing 1 best trials
Trial summary
Hyperparameters:
conv_fixed_filter: 52
conv_fixed_kernel: 3
pool_size: 3
num_cnn_layers: 4
conv_1_filter: 288
conv_1_kernel: 3
conv_2_filter: 208
conv_2_kernel: 3
conv_3_filter: 208
conv_3_kernel: 5
conv_4_filter: 96
conv_4_kernel: 3
num_dense_layers: 1
units_1: 500
activation: relu
dropout: False
rate_1: 0.75
units_2: 180
rate_2: 0.25
lr: 0.0003623815491328145
dropout_1: True
Score: 0.7609999775886536


In [159]:
best_model_cnn_class.evaluate(X_test, y_test)
best_model_cnn_class.save("task2_best_cnn_classification.h5")

94/94 [==============================] - 1s 7ms/step - loss: 0.4735 - accuracy: 0.8707 - clock_loss_reg_mae: 0.0068


In [18]:
untrained_model_cnn_class = best_model_cnn_class
layer_names = []
for layer in best_model_cnn_class.layers:
    name = layer.name
    if 'pooling' not in name and 'flatten' not in name and 'dropout' not in name:
        layer_names.append(name)

for layer in layer_names:
    untrained_model_cnn_class.get_layer(layer).set_weights([np.random.rand(*best_model_cnn_class.get_layer(layer).get_weights()[0].shape),
    np.ones(*best_model_cnn_class.get_layer(layer).get_weights()[1].shape)])

In [ ]:
history = untrained_model_cnn_class.fit(X_train, y_train, epochs=150, verbose=1, 
                                   validation_data=(X_valid, y_valid),
                                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)], batch_size=150)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
#plt.gca().set_ylim(0,1) # set the vertical range to [0-1]
plt.gca().set_xlim(0,10)
plt.show()